In [ ]:
import locale

import pandas as pd

import src.lexisnexis_parser as lnp
from src.config import PARAM, PATH_DATA_R, PATH_DATA_I

batches = PARAM.lexisnexis.batches
locale.setlocale(locale.LC_ALL, 'nl_NL')

In [ ]:
for batch in batches:
    path_raw = PATH_DATA_R / batch
    df = lnp.docxs_to_df(path_raw)
    df = df.drop_duplicates(subset=['title', 'length'], keep='first')
    df.to_pickle(PATH_DATA_I / f'{batch}.pkl')

    # separate section and page
    df = lnp.split_page_from_section(df)

    # extract length as integer
    df['length'] = df.length.str.split(' ').str[0].astype('int')

    # add body as string
    df['body_'] = df.body.str.join('\n')

    # convert date strings to dates
    df['load_date'] = pd.to_datetime(df['load_date'])
    df['publication_date'] = lnp.parse_datestring(
        df['publication_date'],
        split_on=',',
        format='%d %B %Y %A',
        )

    df = lnp.standardize_df(df, batch)
    df.to_pickle(PATH_DATA_I / f'{batch}_.pkl')